In [1]:
# importing libraries
import pandas as pd
import numpy as np
import scipy
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
# method for pca fit and transformation
def pca_fit_transform(X, k = 1):
    mean_matrix = np.mean(X, axis = 0)
    X = X - mean_matrix              # centering data
    covariance_matrix = np.cov(X.T)  # covariance matrix    
    eigen_values, eigen_vectors= np.linalg.eig(covariance_matrix)   #eigenvalues and eigenvectors
    eigen_pairs = [(eigen_values[i], eigen_vectors[:,i]) for i in range(eigen_values.shape[0])] # Making a list of (eigen_values, eigen_vectors) tuples
    eigen_pairs.sort(key=lambda x: x[0], reverse=True)    # Sorting tuples in descending order eigen_values

    # obtaining P with its column vectors corresponding to the top k eigenvectors
    P = eigen_pairs[0][1].reshape(eigen_pairs[0][1].shape[0],1)
    for i in range(1,k):
        P = np.hstack((P,eigen_pairs[i][1].reshape(eigen_pairs[i][1].shape[0],1)))
    transformed = X.dot(P)                       # transforming
    return transformed, mean_matrix, P

# method for pca transformation
def pca_transform(X_test, mean_matrix, P):
    X_test = X_test - mean_matrix                # centralizing data
    transformed = X_test.dot(P)                  # transforming
    return transformed

# method for kernel pca fit and transformation
def kpca_fit_transform(X, k = 1, kernel = 'linear'):
    gamma = 0
    N = X.shape[0]
    if kernel == 'linear':
        K = (X).dot(X.T)                                         # linear Kernel matrix
        
    elif kernel == 'rbf':
        squared_euclidean = squareform(pdist(X, 'sqeuclidean'))  # squared Euclidean distance
        square_form = squareform(squared_euclidean)              # square-form distance matrix to vector-form distance vector
        variance = np.var(square_form)                           
        gamma = 1/(2*variance)
        K = np.exp(-gamma * squared_euclidean)                   # rbf Kernel matrix with gamma = 1/(2*variance)
    
    elif kernel == 'polynomial': 
        K = (X.dot(X.T) + 1) ** 2                                # polynomial kernel with c = 1 and d = 2
        
    # Centering K
    one_matrix = np.ones((N,N)) / N
    K = K - one_matrix.dot(K) - K.dot(one_matrix) + one_matrix.dot(K).dot(one_matrix)
    
    eigen_values, eigen_vectors = scipy.linalg.eigh(K)            # getting eigen values and eigen vectors
    
    eigen_vectors = eigen_vectors / (np.sqrt(eigen_values)*np.linalg.norm(eigen_vectors))   # normalizing eigen vectors
    indexes = eigen_values.argsort()[::-1]                        # sorting

    P = eigen_vectors[:, indexes[0: k]]                           # projection matrix
    X_transformed = K.dot(P)                                      # transforming
    return X_transformed, K, P, gamma

# method for kernel pca transformation
def kpca_transform(X_train, X_test, K_train, P, gamma=1, kernel = 'linear'):
    # kernel matrix
    if kernel == 'linear':
        K = X_test.dot(X_train.T)

    elif kernel == 'rbf':
        stack = np.vstack((X_train, X_test))
        distance = squareform(pdist(stack, 'sqeuclidean'))
        distance[distance<0] = 0
        distance =pd.DataFrame(distance)
        distance = np.array(distance.iloc[X_train.shape[0]:, :X_train.shape[0]])
        K = np.exp(-gamma * distance) 

    elif kernel == 'polynomial':
         K = (X_test.dot(X_train.T) + 1) ** 2  
    
    # transforming
    N  = X_train.shape[0]
    Nt = X_test.shape[0]
    N2 = N*N
    one_test = np.ones((Nt,N))
    one_train = np.ones((N,N))
    s = (1/N2)*K.sum()

    K = K - ((1/N)*(K.dot(one_train))) - ((1/N)*(one_test.dot(K_train))) + (((1/N2)*s)*(one_test.dot(one_train)))
    
    transformed = K.dot(P)
    
    return transformed                          

In [3]:
if __name__ == "__main__":   
    # reading in data
    train_data = pd.read_csv("Data/usps.csv", header = None)
    test_data = pd.read_csv("Data/usps.t.csv", header = None)

    # splitting data and labels
    train_X = np.asarray(train_data.drop([0], axis = 1))
    train_y = np.asarray(train_data[0])
    test_X = np.asarray(test_data.drop([0], axis = 1))
    test_y = np.asarray(test_data[0])

    print('Training data has', train_X.shape[0],'samples and', train_X.shape[1],'features.',
          '\nTesting data has', test_X.shape[0],'samples and', test_X.shape[1],'features.')
    
    model = KNeighborsClassifier() #using KNN classifier for classification
    model.fit(train_X,train_y)
    print("\nModel Accuracy before dimension reduction: ",model.score(test_X, test_y))
    
    
    reduced_dim = 64 
    print("\nReducing dimensions from ",train_X.shape[1], " to ",reduced_dim)
    
    # PCA
    X_train_transformed, mean_mat, projection = pca_fit_transform(train_X, k = reduced_dim)
    X_test_transformed = pca_transform(test_X, mean_mat, projection)
    model.fit(X_train_transformed,train_y)
    print("\nModel Accuracy after PCA dimension reduction: ",model.score(X_test_transformed, test_y))

    # KPCA
    kernel_ = "rbf"     # kernel options: linear, rbf, polynomial
    X_train_transformed, k_train, projection, gamma = kpca_fit_transform(train_X, k = reduced_dim, kernel = kernel_)
    X_test_transformed = kpca_transform(train_X, test_X, k_train, projection, gamma=gamma, kernel = kernel_)
    model.fit(X_train_transformed,train_y)
    print("Model Accuracy after KPCA dimension reduction using ", kernel_," kernel: ",model.score(X_test_transformed, test_y))

Training data has 7291 samples and 256 features. 
Testing data has 2007 samples and 256 features.

Model Accuracy before dimension reduction:  0.9501743896362731

Reducing dimensions from  256  to  64

Model Accuracy after PCA dimension reduction:  0.9501743896362731
Model Accuracy after KPCA dimension reduction using  rbf  kernel:  0.924265072247135
